In [1]:
from pathlib import Path

import pytz

from post_processing.utils.fpod_utils import csv_folder, cpod2aplose, fpod2aplose, actual_data, meta_cut_aplose
from post_processing.utils.core_utils import json2df

### Load data
Import your raw FPOD or CPOD data. All files for one site must be stored in the same folder.
You also need to import your metadata file.

In [2]:
pod_files = Path(r"U:\D")
path = csv_folder(pod_files) #Path to your data folder.
json = Path(r"C:\Users\fouinel\Downloads\deployment_kerguelen.json") #Path to your metadata file.

print(path.head())
df_0 = path.dropna()

metadatax = json2df(json_path=json)
metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +  metadatax["campaign.name"].astype(str))

                 File          ChunkEnd  DPM  Nall  MinsOn  \
0  POD3042 file01.CP3  21/10/2019 12:11    0     0       0   
1  POD3042 file01.CP3  21/10/2019 12:12    0     0       0   
2  POD3042 file01.CP3  21/10/2019 12:13    0     0       0   
3  POD3042 file01.CP3  21/10/2019 12:14    0     0       0   
4  POD3042 file01.CP3  21/10/2019 12:15    0     0       0   

            deploy.name  
0  Site D Simone_Phase1  
1  Site D Simone_Phase1  
2  Site D Simone_Phase1  
3  Site D Simone_Phase1  
4  Site D Simone_Phase1  


In [3]:
d_beg_end = actual_data(df_0, metadatax)
d_beg_end.to_csv(r"U:\Deb_Fin_Site D Simone.csv", index=False)

In [5]:
df_1 = df_0[df_0["DPM"] !=0 ]

### APLOSE format
#### *C-POD*
Use cpod2aplose if you are managing C-POD data.

In [6]:
df_aplose = cpod2aplose(df_1, pytz.utc, "Site D Simone", "Commerson")
print(df_aplose.head())

         dataset filename  start_time  end_time  start_frequency  \
0  Site D Simone                    0        60                0   
1  Site D Simone                    0        60                0   
2  Site D Simone                    0        60                0   
3  Site D Simone                    0        60                0   
4  Site D Simone                    0        60                0   

   end_frequency annotation annotator                start_datetime  \
0              0  Commerson      FPOD  2019-10-26T16:21:00.000+0000   
1              0  Commerson      FPOD  2019-10-26T16:24:00.000+0000   
2              0  Commerson      FPOD  2019-10-26T16:36:00.000+0000   
3              0  Commerson      FPOD  2020-03-14T14:38:00.000+0000   
4              0  Commerson      FPOD  2020-03-14T15:22:00.000+0000   

                   end_datetime  is_box           deploy.name  
0  2019-10-26T16:22:00.000+0000       0  Site D Simone_Phase1  
1  2019-10-26T16:25:00.000+0000     

#### *F-POD*
Use fpod2aplose if you are managing F-POD data.

In [3]:
df_aplose = fpod2aplose(df_1, pytz.utc, "CETIROISE", "Marsouin")
print(df_aplose.head())

     dataset filename  start_time  end_time  start_frequency  end_frequency  \
0  CETIROISE                    0        60                0              0   
1  CETIROISE                    0        60                0              0   
2  CETIROISE                    0        60                0              0   
3  CETIROISE                    0        60                0              0   
4  CETIROISE                    0        60                0              0   

  annotation annotator                start_datetime  \
0   Marsouin      FPOD  2023-02-07T05:34:00.000+0000   
1   Marsouin      FPOD  2023-02-07T05:35:00.000+0000   
2   Marsouin      FPOD  2023-02-07T05:36:00.000+0000   
3   Marsouin      FPOD  2023-02-07T22:40:00.000+0000   
4   Marsouin      FPOD  2023-02-24T15:32:00.000+0000   

                   end_datetime  is_box      deploy.name  
0  2023-02-07T05:35:00.000+0000       0  Point C_Phase 4  
1  2023-02-07T05:36:00.000+0000       0  Point C_Phase 4  
2  2023-02-

### Clean your dataset
Remove useless lines, recorder outside the instrument submersion. Export your file to the aplose format. You can change the name of the file to match the project you are working on.

In [7]:
cleared = meta_cut_aplose(df_aplose, metadatax)
cleared.to_csv(r"U:\APLOSE_D_pos.csv", index=False)